# <img style="float: left; padding: 0px 10px 0px 0px;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg/1920px-Escudo_de_la_Pontificia_Universidad_Cat%C3%B3lica_de_Chile.svg.png"  width="80" /> MCD3100 - Ciencia de Datos Geoespaciales
**Pontificia Universidad Católica de Chile**<br>
**Magister en Ciencia de Datos**<br>

# Tutorial N°8: Análisis Exploratorio de Datos Espaciales.


## 1. Introducción:  Accidentes del tránsito en la ciudad de Santiago.

Como en toda gran ciudad capital, uno de los principales problemas de Santiago es la alta congestión vehicular, y el riesgo de choques y accidentes que ésta conlleva. En este ejercicio, realizaremos un análisis explotatorio de la distribución de choques en la ciudad de Santiago, con el objetivo de identificar y visualizar las zonas de la ciudad con mayor incidencia de choques, y evaluar posibles patrones de autocorrelación global.


## 2. Datos del problema.

En Chile, la Comisión Nacional de Seguridad del Tránsito (CONASET) publica anualmente bases de datos geocodificadas de los siniestros de tránsito registrados en las comunas del área del Gran Santiago en la Región Metropolitana de Santiago. Estas capas contienen detalles de fecha, hora, tipo de accidente, causa basal del accidente, dirección donde ocurrió el accidente, cantidad de personas fallecidas y lesionadas según gravedad, entre otros.

Para el análisis de la distribución de accidentes del tránsito, se utilizarán los conjuntos de datos descritos a continuación.

### 2.1. Cartografía censal de la región Metropolitana: `Cartografía_censo2024_R13.gdb`

Esta cartografía contiene las capas de región, provincias, comunas, límites urbanos, zonas, manzanas, etc. de la región metropolitana.

### 2.2 Siniestros de tránsito, Gran Santiago, RM, Chile, 2024: `Siniestros_urbanos_Metropolitana_2024.zip`

Esta capa contiene la geocodificación de los siniestros de tránsito registrados en las comunas del área del Gran Santiago en la Región Metropolitana de Santiago durante el año 2024. Contiene detalles de fecha, hora, tipo de accidente, causa basal del accidente, dirección donde ocurrió el accidente, cantidad de personas fallecidas y lesionadas según gravedad, entre otros.
Esta base de datos contiene algunos accidentes que no están georreferenciados, los cuales pueden ser eliminados del análisis.


## 3. Desarrollo.

El objetivo de este ejercicio es explorar los datos de siniestros del tránsito en el Gran Santiago para obtener una mejor comprensión de la distribución espacial de estos eventos, identificar las zonas de mayor incidencia de accidentes, y posibles patrones espaciales en su ocurrencia. Para ello, se proponen los siguientes pasos:

- Graficar los puntos de ocurrencia de los accidentes del tránsito junto con la cartografía del Gran Santiago.
- Visualizar los datos agregados por zona censal.
- Visualizar los datos agregados en un agrilla regular.
- Generar un mapa de kernel de densidad.
- Evaluar si existe un patrón de autocorrelación espacial en la ocurrencia de accidentes.


In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### 3.1 Visualización preliminar de los datos.

Para una primera exploración de los datos, graficaremos las ubicaciones de los siniestros 2024 en la Región Metropolitana junto a algunas capas de la cartografía censal: comunas, zonas y límite urbano.

In [ ]:
#Leemos los datos de siniestros 2024

st=gpd.read_file('Siniestros_urbanos_Metropolitana_2024/Siniestros_urbanos_Metropolitana_2024.shp')
st.head()

In [ ]:
st.plot();

In [ ]:
#lista de capas de la cartografía censal
gpd.list_layers('Cartografía_censo2024_R13.gdb/Cartografía_censo2024_R13.gdb')

In [ ]:
#Seleccionaremos algunas capas de referencia: zonas censales
zonas=gpd.read_file('Cartografía_censo2024_R13.gdb/Cartografía_censo2024_R13.gdb',layer='Zonal_CPV24')

#Definimos el crs de referencia para todos los gráficos
crs=zonas.crs
crs

In [ ]:
zonas.LOCALIDAD

In [ ]:
#Seleccionamos las zonas urbanas correspondientes al Gran Santiago
zonas=zonas[zonas['LOCALIDAD']=='GRAN SANTIAGO']

#Seleccionamos sólo las columnas necesarias
zonas=zonas[['ID_ZONA','geometry']]
zonas['ID_ZONA']=zonas['ID_ZONA'].astype('int')

In [ ]:
zonas.plot()

In [ ]:
st=st.to_crs(crs)
st=gpd.overlay(st,zonas[['geometry']])

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)

st.to_crs(crs).plot(ax=ax,marker='.',legend=True,color='green',label='Siniestros 2024')
zonas.to_crs(crs).boundary.plot(ax=ax,color='k',lw=0.5,label='Zonas censales');
ax.legend();

In [ ]:
st.head()

Para mejor visualización y exploración, podemos ajustar el tamaño y transparencia de los marcadore (puntos):

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)

zonas.boundary.plot(ax=ax,color='gray',lw=0.2,label='Zonas')
st.plot(ax=ax,marker='.',legend=True,color='green',label='Siniestros 2024',markersize=3,alpha=0.7)

ax.legend();

### 3.2 Visualización de accidentes agregados por zona censal.

Agregaremos los datos de siniestros de acuerdo a la división en zonas de la cartografía censal.

In [ ]:
st.head()

In [ ]:
zonas_st=gpd.sjoin(zonas,st,how='inner')
zonas_st

Al hacer el join espacial entre la zonas cenales y los siniestros del tránsito, las zonas se repiten tantas veces como accidentes hayan ocurrido dentro de ellas. Por lo tanto, ahora es necesario agregar este GeoDataFrame por zonas, usando la columna **ID_ZONA**. Para esto, podemos usar por ejemplo una tabla pivote de pandas.

In [ ]:
pv=pd.pivot_table(zonas_st[['ID_ZONA','geometry']],index='ID_ZONA',aggfunc='count').reset_index()
pv

In [ ]:
pv.rename(columns={'geometry':'N_siniestros'},inplace=True)
pv

In [ ]:
zonas_count=pd.merge(zonas,pv,on='ID_ZONA')
zonas_count

Finalmente, generamos el gráfico de coropleta.

In [ ]:
fig=plt.figure(figsize=(8,8))
ax=fig.add_subplot(111)

zonas_count.plot(ax=ax,column='N_siniestros',cmap='Reds',scheme='natural_breaks',legend=True,edgecolor='k',lw=0.1)
ax.set_xlim(-71,-70.4)
ax.set_ylim(-33.7,-33.2);

La forma en que se escalan los mapas de color también se puede manipular con la opción `scheme` (requiere tener instalada la librería  `mapclassify`). La opción se puede configurar con cualquier esquema proporcionado por mapclassify, como por ejemplo:'box_plot', 'equal_interval', 'fisher_jenks', 'fisher_jenks_sampled', 'headtail_breaks', 'jenks_caspall', 'jenks_caspall_forced', 'jenks_caspall_sampled', 'max_p_classifier', 'maximum_breaks', 'natural_breaks', 'quantiles', 'percentiles', 'std_mean' o 'user_defined'.

 Consulte la documentación de [geopandas](https://geopandas-org.translate.goog/en/stable/docs/user_guide/mapping.html?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc) y [mapclassify](https://pysal.org/mapclassify/) para obtener más detalles sobre estos esquemas de clasificación de mapas.



### 3.2 Visualización de accidentes en una grilla hexagonal.

Si no contamos con una capa de polígonos de rerefencia relevantes para el análisis exploratorio de datos espaciales, también es posible agregar los puntos en polígonos regulares utilizando una grilla regular definida por el usuario, ya sea cuadrada o hexagonal. La ventaja de utilizar una grilla regular, es que cada celda tiene la misma área, por lo que el gráfico de coropletas equivale a un mapa de densidad de puntos.

#### Grilla global H3.

Una herramienta útil para generar mapas de densidad es el sistema global de cuadrícula geoespacial de código abierto **H3** de Uber, que cubre toda la Tierra con mosaicos repetitivos. El componente básico del sistema de cuadrícula es un hexágono y se puede elegir entre 16 tamaños diferentes de hexágonos que varían desde el área de un país grande hasta el área de una mesa auxiliar pequeña.

Puede encontrar la información detallada de [H3 en este link](https://h3geo-org.translate.goog/?_x_tr_sl=en&_x_tr_tl=es&_x_tr_hl=es&_x_tr_pto=tc&_x_tr_hist=true).

El sistema H3 es de código abierto y existen APIs de Python y varios otros lenguajes para trabajar con ella y usarla en análisis geoespaciales. Por ejemplo, la función `h3fy`del módulo `tobler.util` de PySAL, permite generar fácilmente una grilla hexagonal sobre el área cubierta por un geodataframe:

In [ ]:
from tobler.util import h3fy


In [ ]:
rm_hex = h3fy(zonas_count[['geometry']],resolution=8)

fig=plt.figure(figsize=(8,8))
ax=fig.add_subplot(111)
rm_hex.boundary.plot(ax=ax)

In [ ]:
rm_hex

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)

#zonas_count.plot(ax=ax,column='N_siniestros',cmap='Reds',legend=True,edgecolor='white',lw=0.1)
st.plot(ax=ax,markersize=1,alpha=0.7)
rm_hex.boundary.plot(ax=ax,lw=0.1,color='k')


In [ ]:
hex_st=gpd.sjoin(rm_hex,st,how='inner')
pv=pd.pivot_table(hex_st,index='hex_id',aggfunc='count').reset_index()
pv.rename(columns={'geometry':'N_siniestros'},inplace=True)
hex_count=pd.merge(rm_hex,pv,on='hex_id')
hex_count

In [ ]:
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(111)

#zonas_count.plot(ax=ax,column='N_siniestros',cmap='Reds',legend=True,edgecolor='white',lw=0.1)
rm_hex.boundary.plot(ax=ax,lw=0.1,color='k')
hex_count.plot(ax=ax,column='N_siniestros',cmap='Reds',scheme='natural_breaks',legend=True);

### 3.3 Mapa de kernel de densidad.

Finalmente, podemos visualizar la distribución de siniestros como un mapa de kernel de densidad, o **KDE**. Ésta es una aproximación empírica de la función de densidad de probabilidad.

En lugar de superponer una grilla de cuadrados o hexágonos y contar cuántos puntos caen dentro de cada celda, un KDE dispone una grilla de puntos sobre el espacio de interés, sobre los cuales coloca funciones núcleo (kernel) que contabilizan los puntos cercanos asignándoles un peso diferente en función de la distancia. Luego, estos conteos se agregan para generar una superficie global de probabilidad. La función núcleo más común es la gaussiana, que aplica una distribución normal para ponderar los puntos. El resultado es una superficie continua con una función de probabilidad que puede evaluarse en cada punto del espacio.

Crear un mapa de kernel gaussiano en Python es bastante sencillo, utilizando la función [seaborn.kdeplot()](https://seaborn.pydata.org/generated/seaborn.kdeplot.html).

In [ ]:
import seaborn as sns

fig=plt.figure(figsize=(8,8))
ax1=fig.add_subplot(111,xlabel='Longitud',ylabel='Latitud')


#hex_count.plot(ax=ax1,column='N_siniestros',cmap='Reds');
sns.kdeplot(ax=ax1,
    x=st.geometry.x,
    y=st.geometry.y,
    fill=True,
    cmap="Reds",levels=20,lw=0.5,legend=True,cbar=True);




### 3.4 Análisis de Autocorrelación espacial.

Los mapas anteriores muestran ciertas zonas de sobredensidad, o aglomeraciones de siniestros del tránsito en ciertas zonas del Gran Santiago. Sin embargo, ¿significa esto que existe un patrón estadísticamente válido, que nos permita asegurar que las zonas con mayor incidencia de siniestros están espacialmente agrupadas?
Para responder esta pregunta, podemos evaluar el índice de autocorrelación global de los datos. Si hay una autocorrelación positiva, quiere decir que existen zonas caracaterizadas por un alta incidencia de choques, que son cercanas entre sí, y por lo tanto forman "zonas críticas" en cuanto a siniestros del tránsito.

In [ ]:
zonas_count.head()

In [ ]:
#Usamos pysal para obtener una matriz de pesos espaciales, por ejemplo Queen.
from libpysal import weights
w = weights.contiguity.Queen.from_dataframe(zonas_count,idVariable='ID_ZONA')
w.transform ='R'


In [ ]:
y=zonas_count['N_siniestros']
#Normalizamos la variable Y
mean=y.mean()
std=y.std()
y_n=(y-mean)/std

#Lag espacial
y_lag = weights.lag_spatial(w, y_n)

zonas_count['y_n']=y_n
zonas_count['y_lag']=y_lag

In [ ]:
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

fig=plt.figure(figsize=(15,5))
ax1=fig.add_subplot(131)
ax2=fig.add_subplot(132)
ax3=fig.add_subplot(133)

ax_divider = make_axes_locatable(ax1)
cax1 = ax_divider.append_axes("right", size="4%", pad="1%")

ax_divider = make_axes_locatable(ax2)
cax2 = ax_divider.append_axes("right", size="4%", pad="1%")


zonas_count.plot(ax=ax1,column='y_n',cmap='Reds',legend=True,edgecolor='k',lw=0.1,cax=cax1)
ax1.set_title('Y_norm')

zonas_count.plot(ax=ax2,column='y_lag',cmap='Reds',legend=True,edgecolor='k',lw=0.1,cax=cax2)
ax2.set_title('Y_lag')

for ax in [ax1,ax2]:
    ax.set_xlim(-70.9,-70.4)
    ax.set_ylim(-33.7,-33.25);
    ax.set_xticks([])
    ax.set_yticks([])


ax3.plot(zonas_count['y_n'],zonas_count['y_lag'],marker='.',lw=0)
ax3.set_xlim(-1,5)
ax3.set_ylim(-1,5)
ax3.axline((0, 0), slope=1, color='r', linestyle='--')
ax3.set_aspect('equal')
ax3.set_title('Gráfico de Moran');



In [ ]:
#Calculemos el valor de I global
import libpysal
from esda.moran import Moran

mi = Moran(zonas_count['y_n'], w)
print(mi.I)

De acuerdo a nuestro análisis exploratorio de datos, si bien hay zonas con mayor incidencia de accidentes del tránsito, no se observa un patrón de autocorrelación espacial.
